# 03 - Philips PAR/REC Input

This notebook demonstrates how to use the `diffusemri` library to handle Philips PAR/REC image files. PAR/REC is a format used by Philips MRI scanners, where the `.par` file contains header information and the `.rec` file contains the raw image data.

The `diffusemri` library leverages `nibabel` for its PAR/REC reading capabilities. `nibabel` typically supports PAR/REC versions 4.0, 4.1, and 4.2. This notebook will show how to:

1.  Read data from PAR/REC files.
2.  Convert PAR/REC files (including potential DWI information) to NIfTI format.

In [ ]:
import os
import shutil
import numpy as np
import nibabel as nib

# diffusemri library imports
from data_io.parrec_utils import read_parrec_data, convert_parrec_to_nifti

# For conceptual CLI examples, one might import the main function if designed for it:
# from cli.run_format_conversion import main as conversion_cli_main 

# Setup a temporary directory for example files
TEMP_DIR = "temp_parrec_example"
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR)

print(f"Temporary directory for examples: {os.path.abspath(TEMP_DIR)}")

## Part 1: Reading PAR/REC Data

### Important Note on PAR/REC Files for This Notebook

PAR/REC files are complex and contain scanner-specific binary data. Creating fully valid and representative dummy PAR/REC files programmatically within a notebook is extremely challenging, especially for DWI data with all its associated metadata.

**For the following examples to be truly meaningful, you should replace the placeholder `dummy_par_filepath` with the actual path to one of your own `.par` or `.rec` files.**

We will create a highly simplified, minimal set of `dummy.par` and `dummy.rec` files. These are primarily to allow the code cells to run without immediate error. However, they **may not be fully parsable by `nibabel` or reflect the true structure and content of real PAR/REC data.** DWI information, in particular, will be absent or incomplete in these dummies.

In [ ]:
# Create a very minimal dummy .par file content
# This header is extremely simplified and may lack many fields needed by nibabel for full parsing.
dummy_par_content = """
# === DATA DESCRIPTION FILE ==================================================
# Dataset name: dummy_parrec_data_for_notebook
# === GENERAL INFORMATION ====================================================
.    Patient name                       : P_Notebook_Dummy
.    Examination name                   : E_Notebook_Dummy
.    Protocol name                      : Proto_Notebook_Dummy
.    Examination date/time              : 2023.01.01 / 12:00:00
.    Series Type                        : Image
.    Acquisition NR                     : 1
.    Reconstruction NR                  : 1
.    Scan Duration [sec]                : 60
.    Max. number of cardiac phases      : 1
.    Max. number of echoes              : 1
.    Max. number of slices/locations    : 1
.    Max. number of dynamics            : 1
.    Max. number of mixes               : 1
.    Patient position                   : Head First Supine
.    Preparation direction              : Anterior-Posterior
.    Technique                          : SE
.    Scan resolution  (x, y)            : 2   2
.    Scan mode                          : 2D
.    Repetition time [ms]               : 1000.0
.    FOV (ap, fh, rl) [mm]              : 4.0   4.0   5.0
.    Slice thickness [mm]               : 5.0
.    Slice gap [mm]                     : 0.0
.    Water Fat shift [pixels]           : 0.0
.    Angulation midslice(ap,fh,rl)[degr]: 0.0   0.0   0.0
.    Off Centre midslice(ap,fh,rl) [mm] : 0.0   0.0   0.0
.    Slice orientation (TRA/SAG/COR)    : TRANSVERSAL
.    Phase encoding velocity [cm/sec]   : 0.0 0.0 0.0
.    MTC                                : NO
.    SPIR                               : NO
.    EPI factor                         : 1
.    Dynamic scan                       : NO
.    Diffusion                          : NO
.    Diffusion echo time [ms]           : 0.0
# === IMAGE INFORMATION DEFINITION ===========================================
# The rest of this file contains ONE line per image, this line contains the following information:
# 
#  slice number                             (integer)
#  echo number                              (integer)
#  dynamic scan number                      (integer)
#  cardiac phase number                     (integer)
#  image_type_mr                            (integer)
#  scanning sequence                        (integer)
#  index in REC file (in images)            (integer)
#  image pixel size (in bits)               (integer)
#  scan percentage                          (integer)
#  image_angulation_ap                      (float)
#  image_angulation_fh                      (float)
#  image_angulation_rl                      (float)
#  image_offcentre_ap                       (float)
#  image_offcentre_fh                       (float)
#  image_offcentre_rl                       (float)
#  slice_thickness                          (float)
#  slice_gap                                (float)
#  image_display_orientation                (integer)
#  slice_orientation                        (integer)
#  fmri_status_indication                   (integer)
#  image_type_ed_es                         (integer)
#  pixel_spacing_x_y                        (float) (float)
#  echo_time                                (float)
#  dyn_scan_begin_time                      (float)
#  trigger_time                             (float)
#  diffusion_b_factor                       (float)
#  number_of_averages                       (integer)
#  image_flip_angle                         (float)
#  cardiac_frequency                        (integer)
#  minimum_RR_interval                      (integer)
#  maximum_RR_interval                      (integer)
#  TURBO_factor                             (integer)
#  diffusion_b_value_number                 (integer)
#  gradient_orientation_number              (integer)
#  contrast_type                            (string)
#  diffusion_anisotropy_type                (string)
#  diffusion_ap                             (float)
#  diffusion_fh                             (float)
#  diffusion_rl                             (float)
# === IMAGE INFORMATION ======================================================
#  sl ec dyn ph ty    idx pix scan%   ap_ang   fh_ang   rl_ang   ap_off   fh_off   rl_off  thick    gap     disp_orien     slice_orien  fmri_stat type_ed_es pixel_spacing_xy    TE      dyn_time trig_time b_factor   avgs  flip_angle   card_freq min_RR max_RR turbo_fac   b_val_num   grad_num   contr_type   diff_anis_type   diff_ap  diff_fh  diff_rl
   1  1  1  1  0     0     0  16  100     0.000    0.000    0.000    0.000    0.000    0.000   5.000    0.000      0               2            0            0            2.000 2.000   0.0      0.0       0.0       0.0        1    90.0         0         0        0         0           0           0          DEFAULT      NONE             0.000    0.000    0.000
"""
dummy_par_filepath = os.path.join(TEMP_DIR, "dummy.par")
with open(dummy_par_filepath, 'w') as f:
    f.write(dummy_par_content)

# Create a dummy .rec file: 2x2 pixels, 1 slice, uint16 data
# Image pixel size (bits) = 16 -> 2 bytes per pixel
# Scan resolution (x, y) = 2 2 -> 2x2 = 4 pixels per slice
# Total bytes = 4 pixels * 2 bytes/pixel = 8 bytes
dummy_rec_data = np.array([[100, 200], [300, 400]], dtype=np.uint16).tobytes()
dummy_rec_filepath = os.path.join(TEMP_DIR, "dummy.rec")
with open(dummy_rec_filepath, 'wb') as f:
    f.write(dummy_rec_data)

print(f"Created dummy PAR file: {dummy_par_filepath}")
print(f"Created dummy REC file: {dummy_rec_filepath}")

In [ ]:
print("--- Attempting to read PAR/REC data ---")
print("Note: This uses the highly simplified dummy PAR/REC files.")
print("For actual use, replace 'dummy_par_filepath' with a path to your real PAR or REC file.")

try:
    # Provide the path to the .par file; nibabel will find the .rec file
    image_data, affine, bvals, bvecs, metadata_dict = read_parrec_data(dummy_par_filepath)
    
    if image_data is not None:
        print("\nSuccessfully read PAR/REC data (or at least parts of it):")
        print(f"  Image data shape: {image_data.shape} (Expected based on dummy: (2, 2, 1))")
        print(f"  Image data type: {image_data.dtype}")
        print(f"  Affine matrix:\n{affine}")
        if bvals is not None:
            print(f"  b-values: {bvals}")
        else:
            print("  b-values: Not found (expected for this non-DWI dummy)")
        if bvecs is not None:
            print(f"  b-vectors shape: {bvecs.shape}")
        else:
            print("  b-vectors: Not found (expected for this non-DWI dummy)")
        # print(f"  Metadata dictionary (nibabel header): {metadata_dict}")
    else:
        print("\nReading PAR/REC data returned None for image_data. The dummy files might be too simple or invalid.")

except Exception as e:
    print(f"\nError reading PAR/REC files: {e}")
    print("This error might be expected due to the oversimplified nature of the dummy PAR/REC files.")
    print("Please test with your own valid PAR/REC data.")

## Part 2: PAR/REC to NIfTI Conversion

The `convert_parrec_to_nifti` function reads a PAR/REC file pair and saves the image data as a NIfTI file. If DWI information (b-values and b-vectors) is found in the PAR header, it will also save them to `.bval` and `.bvec` text files respectively, provided output paths for them are given.

In [ ]:
nifti_output_filepath = os.path.join(TEMP_DIR, "parrec_converted.nii.gz")
bval_output_filepath = os.path.join(TEMP_DIR, "parrec_converted.bval")
bvec_output_filepath = os.path.join(TEMP_DIR, "parrec_converted.bvec")

print(f"\n--- Attempting PAR/REC to NIfTI conversion ---")
print("Using the dummy PAR/REC files. This may not fully succeed or extract DWI info.")
print("Replace 'dummy_par_filepath' with your real PAR/REC file path for actual conversion.")

try:
    success = convert_parrec_to_nifti(
        parrec_filepath=dummy_par_filepath,
        output_nifti_file=nifti_output_filepath,
        output_bval_file=bval_output_filepath,
        output_bvec_file=bvec_output_filepath
    )
    
    if success:
        print(f"\nConversion function executed. Check existence of output files:")
        if os.path.exists(nifti_output_filepath):
            print(f"  NIfTI file created: {nifti_output_filepath}")
            # Optionally, load and inspect the NIfTI image
            # img = nib.load(nifti_output_filepath)
            # print(f"    Loaded NIfTI: shape={img.shape}, affine=\n{img.affine}")
        else:
            print(f"  NIfTI file NOT created: {nifti_output_filepath}")
        
        if os.path.exists(bval_output_filepath):
            print(f"  bval file created: {bval_output_filepath}")
        else:
            print(f"  bval file NOT created (or not applicable for this dummy data).")
            
        if os.path.exists(bvec_output_filepath):
            print(f"  bvec file created: {bvec_output_filepath}")
        else:
            print(f"  bvec file NOT created (or not applicable for this dummy data).")
    else:
        print("\nPAR/REC to NIfTI conversion function returned False or did not complete as expected.")

except Exception as e:
    print(f"\nError during PAR/REC to NIfTI conversion: {e}")
    print("This might be due to the simplified dummy files. Test with real data.")

### PAR/REC to NIfTI (CLI Conceptual Example)

The `diffusemri` library also provides a command-line tool for this conversion. You would typically run this from your terminal.

```bash
# Replace /path/to/... with your actual file paths
python cli/run_format_conversion.py parrec2nii \
    --input_parrec /path/to/your_scan.par \
    --output_nifti /path/to/output/converted.nii.gz \
    --output_bval /path/to/output/converted.bval \
    --output_bvec /path/to/output/converted.bvec \
    # Optional arguments like --scaling_method or --no_strict_sort can also be added
```
*(To run this directly in the notebook, you'd typically use `!python ...` or the `conversion_cli_main` Python function if available and paths are correctly set.)*

## Important Considerations for PAR/REC Handling

When working with PAR/REC files, especially through `nibabel` (which `diffusemri` uses under the hood), keep these points in mind:

*   **`strict_sort` Option**: The `read_parrec_data` and `convert_parrec_to_nifti` functions accept a `strict_sort` parameter (defaults to `True`). This controls how `nibabel` sorts image volumes, particularly for DWI data where scan sequences can be complex. If you encounter issues with volume ordering, you might experiment with `strict_sort=False`, but be cautious as it can lead to incorrect gradient assignments if not understood properly.

*   **`scaling` Option**: PAR/REC files can store pixel values in different ways. The `scaling` parameter (defaults to `'dv'`) in the `diffusemri` functions is passed to `nibabel` to control how these values are interpreted and scaled. 
    *   `'dv'`: Display values. These are values scaled for direct viewing, possibly incorporating modality-specific conversions.
    *   `'fp'`: Floating point values. These might be closer to the raw proportional values.
    *   `'ss'`: Scale slope. Uses a more complex calculation involving rescale slope, intercept, and stored values.
    The appropriate choice can depend on the specific PAR/REC version and how the data was acquired and stored. For quantitative analysis, understanding this scaling is important.

*   **DWI Metadata**: The extraction of b-values and b-vectors relies on `nibabel`'s ability to parse this information from the `.par` file header. If the PAR file does not contain standard fields for DWI information, or if it's in a non-standard format, this information may not be extracted correctly or at all.

*   **File Pair**: Always ensure that both the `.par` (header) and `.rec` (data) files are present in the same directory. `nibabel` (and therefore `diffusemri`) expects to find the corresponding `.rec` file when given a `.par` file, and vice-versa.

## Cleanup

Remove the temporary directory and its contents.

In [ ]:
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
    print(f"Cleaned up temporary directory: {TEMP_DIR}")